# Download STAR and HTSeq - only for google colab

In [ ]:
#%%bash

#wget https://github.com/alexdobin/STAR/raw/master/bin/Linux_x86_64_static/STAR
#chmod a+x STAR

In [ ]:
#%%bash

#pip install HTSeq


# Load fastq data

This is a toy example, only includes chr1

In [ ]:
%%bash
wget https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/fastq_small/SRR3051643_1.fastq.gz
wget https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/fastq_small/SRR3051643_2.fastq.gz

wget https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/fastq_small/SRR3051648_1.fastq.gz
wget https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/fastq_small/SRR3051648_2.fastq.gz

In [3]:
%%bash
gunzip *.fastq.gz

# Building genome index for STAR

We will only include chromosome 1 (otherwise, STAR will require 30Gb of RAM, it's more than colab offers). Remove this parameter to work with the whole genome: --genomeSAindexNbases 12 

In [ ]:
%%bash

mkdir genome/
mkdir genome/mm10/

#load genome sequence - only chromosome 1
wget -O genome/mm10/chr1.fa.gz http://hgdownload.cse.ucsc.edu/goldenpath/mm10/chromosomes/chr1.fa.gz
gunzip genome/mm10/chr1.fa.gz

In [ ]:
%%bash

#gene annotation - only chromosome 1
wget https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/genes.mm10_chr.chr1.gtf

In [7]:
%%bash

#build index

mkdir genome/mm10/star
STAR --runThreadN 4 --limitGenomeGenerateRAM 10000000000 --genomeSAindexNbases 12 --runMode genomeGenerate --genomeDir genome/mm10/star --genomeFastaFiles genome/mm10/chr1.fa

Dec 14 02:33:51 ..... started STAR run
Dec 14 02:33:51 ... starting to generate Genome files
Dec 14 02:33:55 ... starting to sort Suffix Array. This may take a long time...
Dec 14 02:33:57 ... sorting Suffix Array chunks and saving them to disk...
Dec 14 02:36:50 ... loading chunks from disk, packing SA...
Dec 14 02:36:54 ... finished generating suffix array
Dec 14 02:36:54 ... generating Suffix Array index
Dec 14 02:37:07 ... completed Suffix Array index
Dec 14 02:37:07 ... writing Genome to disk ...
Dec 14 02:37:07 ... writing Suffix Array to disk ...
Dec 14 02:37:08 ... writing SAindex to disk
Dec 14 02:37:08 ..... finished successfully


mkdir: genome/mm10/star: File exists


# Mapping with STAR

In [9]:
%%bash
mkdir data/bam/
STAR --genomeDir genome/mm10/star --readFilesIn  SRR3051643_1.fastq SRR3051643_2.fastq --runThreadN 4 --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 10000000000 \
    --outFileNamePrefix data/bam/SRR3051643.bam

Dec 14 02:37:23 ..... started STAR run
Dec 14 02:37:23 ..... loading genome
Dec 14 02:37:25 ..... started mapping
Dec 14 02:37:40 ..... finished mapping
Dec 14 02:37:40 ..... started sorting BAM
Dec 14 02:37:40 ..... finished successfully


mkdir: data: No such file or directory


In [11]:
%%bash
mkdir data/bam/
STAR --genomeDir genome/mm10/star --readFilesIn  SRR3051648_1.fastq SRR3051648_2.fastq --runThreadN 4 --outSAMtype BAM SortedByCoordinate --limitBAMsortRAM 10000000000 \
    --outFileNamePrefix data/bam/SRR3051648.bam

Dec 14 02:38:00 ..... started STAR run
Dec 14 02:38:00 ..... loading genome
Dec 14 02:38:01 ..... started mapping
Dec 14 02:38:18 ..... finished mapping
Dec 14 02:38:18 ..... started sorting BAM
Dec 14 02:38:19 ..... finished successfully


mkdir: data/bam/: File exists


Workaround, in case you don't have enough RAM to run STAR:

In [ ]:
#%%bash

#wget -O SRR3051643.bamAligned.sortedByCoord.out.bam https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/bam/SRR3051643.bamAligned.sortedByCoord.out.bam
#wget -O SRR3051648.bamAligned.sortedByCoord.out.bam https://adameykolab.srv.meduniwien.ac.at/share/teaching/bulkRNAseq/bam/SRR3051648.bamAligned.sortedByCoord.out.bam

In [1]:
%%bash
#indexing bams
samtools index data/bam/SRR3051643.bamAligned.sortedByCoord.out.bam
samtools index data/bam/SRR3051648.bamAligned.sortedByCoord.out.bam

# Count reads per sample per gene

In [5]:
%%bash

htseq-count -f bam -t gene -i gene_name  \
    data/bam/SRR3051643.bamAligned.sortedByCoord.out.bam \
    data/bam/SRR3051648.bamAligned.sortedByCoord.out.bam \
    genes.mm10_chr.chr1.gtf > data/counts.tab

100000 GFF lines processed.
108828 GFF lines processed.
100000 alignment record pairs processed.
200000 alignment record pairs processed.
300000 alignment record pairs processed.
400000 alignment record pairs processed.
500000 alignment record pairs processed.
520924 alignment pairs processed.
100000 alignment record pairs processed.
200000 alignment record pairs processed.
300000 alignment record pairs processed.
400000 alignment record pairs processed.
500000 alignment record pairs processed.
587894 alignment pairs processed.
